<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_KoSpacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 띄어쓰기 검사 모델 구현하기

## 필요한 패키지 다운로드

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!pip install kss

In [36]:
pip install sentencepiece

In [37]:
pip install utils

In [38]:
pip install pytorch_lightning

In [39]:
pip install omegaconf

In [40]:
pip install seqeval

In [41]:
pip install tokenizers

In [42]:
pip install transformers

## 데이터 불러오기

In [ ]:
import os

root = '/content/drive/MyDrive/news_class9x1400/'
categories = os.listdir(root)

dataset = []

for cat in categories:
  files = os.listdir(root + cat)
  for i, f in enumerate(files):
    fname = root + cat + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()

print(len(dataset))

12600


In [ ]:
import random

random.shuffle(dataset)
mini_dataset = dataset[:800]

In [ ]:
mini_dataset[241]

['정세균 유명환 장관 욕설, 기절초풍할 상황\n민주당이 지난 22일 외통위에서 천정배 의원을 가리키며 미친X 욕설을 한 유명환 외교통상부장관의 경질을 요구하고 나섰다.\n정세균 대표는 29일 오전 국회 본관 246호에서 열린 의원총회 모두발언을 통해 속된 말로 기절초풍할 상황이 대한민국 국회에서 벌어졌다며 국회 경시하는 풍조가 벌써 몇 번째냐, 총리부터 시작해서 문화부 장관, 기획재정부 장관, 거기에 외교부 장관까지 가세를 하는 형국이 됐다고 비판했다.\n정 대표는 또 사적인 술자리에서 그런 얘기를 해도 용납될 수 없는 일인데, 국회에 장관이 나와서 국무위원석에 앉아서 야당 중진의원에게 그런 얘기를 했다는 것이 기가 막힐 노릇이라며 절대 용납해서는 안 된다고 목소리를 높였다.\n이어 그는 한승수 총리가 공개적으로 사과하고 유명환 외교부 장관은 스스로 사퇴해야 한다며 만약 사퇴하지 않으면 대통령이 경질해야 한다고 주장했다.\n아울러 정 대표는 각 상임위에서 정부 각료들의 버릇을 뜯어고쳐야 한다며 민주당 의원들에게 강력한 대응을 요청했다.\n그는 입법부는 국민의 대표가 나와서 국민을 대신해서 일을 하고 있는 것인데, 장관들이 어떻게 이런 짓을 할 수 있느냐면서 이 부분에 대해서 확실하게 노력해 주시기 바란다고 말했다.\n노영민 대변인도 공식 브리핑을 통해 유명환 장관을 강력히 비난했다.\n노 대변인은 유 장관을 향해 국무위원의 발언이라고는 생각하기 어려운 시정잡배 같은 막말이라고 비판한 뒤 국회를 없애야 한다는 위험천만한 발언을 국민이 어떻게 받아들일지 걱정이라고 지적했다.\n노 대변인은 또 도대체 이 정부 국무위원들의 수준이 왜 이런지 모르겠다면서 유유상종이라는 말도 있고, 윗물이 맑아야 아랫물이 맑다는 얘기도 있다, 이명박 대통령은 제발 주변 좀 돌아보시기 바란다고 성토했다.\n박병석 정책위원장도 이날 오전 <불교방송> 김재원의 아침저널에 출연해 유 장관의 발언에 대해 분노와 자괴감이 든다면서 한나라당이 청와대의 지시와 방침에 맹종하기 때문에 이런 경시 문제

## 데이터 전처리

In [ ]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': ''

})
test = '"안녕"하세★요? 좋은 아▲침입┌니├다. 오│늘도 힘내└세요→!'
test = test.translate(table)

test

'안녕하세요? 좋은 아침입니다. 오늘도 힘내세요!'

In [ ]:
import kss
import random

process_dataset = []

for sentences in mini_dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split("\n")

  for sentence in sentences:
    process_dataset.extend(kss.split_sentences(sentence))


len(process_dataset)

31800

In [ ]:
# 4h 36m 

len(process_dataset)

31800

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(process_dataset, test_size = 0.2, random_state  = 42)
train, val = train_test_split(train, test_size = 0.2, random_state = 42)

print(len(train), len(val), len(test))

20352 5088 6360


In [ ]:
def save_datafile(file_name, dataset):
  with open(
      os.path.join(root, file_name), mode = 'w', encoding='utf-8'
  ) as f:
    for data in dataset:
      f.write(data)

save_datafile('train.txt', train)
save_datafile('val.txt', val)
save_datafile('test.txt', test)

print('train, val, test dataset 저장 완료!')

train, val, test dataset 저장 완료!


In [ ]:
train[11901]

'올 시즌의 브랜드는 자신과 같이 트윈타워를 이루는 하승진을 잘 이용해주고 있다.'

In [ ]:
val[3452]

'그리하여 늘 양심을 생각하면서'

In [ ]:
test[999]

'말글 얼거리를 잘 모르는 어리보기가 국어사전을 엮었다면 모르되, 우리 말글 얼거리를 깊이 헤아리고 익히고 가다듬는다는 국어학자가 이렇게 국어사전을 엮었다면 더없이 슬프고 안타까운 노릇입니다.'

## 띄어쓰기를 위한 클래스

In [43]:
from typing import List

def load_slot_labels() -> List[str]:
    """tag label 종류 리턴"""
    return ["UNK", "PAD", "O", "B", "I", "E", "S"]

In [44]:
from typing import Callable, List, Tuple
from torch.utils.data import Dataset

class CorpusDataset(Dataset):
    def __init__(self, data_path: str, transform: Callable[[List, List], Tuple]):
        self.sentences = []
        self.transform = transform
        self.slot_labels = load_slot_labels()

        self._load_data(data_path)

    def _load_data(self, data_path: str):
        """data를 file에서 불러온다.
        Args:
            data_path: file 경로
        """
        with open(data_path, mode="r", encoding="utf-8") as f:
            lines = f.readlines()
            self.sentences = [line.split() for line in lines]

    def _get_tags(self, sentence: List[str]) -> List[str]:
        """문장에 대해 띄어쓰기 tagging을 한다.
        character 단위로 분리하여 BIES tagging을 한다.
        Args:
            sentence: 문장
        Retrns:
            문장의 각 토큰에 대해 tagging한 결과 리턴
            ["B", "I", "E"]
        """

        all_tags = []
        for word in sentence:
            if len(word) == 1:
                all_tags.append("S")
            elif len(word) > 1:
                for i, c in enumerate(word):
                    if i == 0:
                        all_tags.append("B")
                    elif i == len(word) - 1:
                        all_tags.append("E")
                    else:
                        all_tags.append("I")
        return all_tags

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = "".join(self.sentences[idx])
        tags = self._get_tags(self.sentences[idx])
        tags = [self.slot_labels.index(t) for t in tags]

        (
            input_ids,
            slot_labels,
            attention_mask,
            token_type_ids,
        ) = self.transform(sentence, tags)

        return input_ids, slot_labels, attention_mask, token_type_ids

### Tokenization_kobert


In [45]:
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBERT model """


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [46]:
import torch

class Preprocessor:
    def __init__(self, max_len: int):
        self.tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
        self.max_len = max_len
        self.pad_token_id = 0

    def get_input_features(
        self, sentence: List[str], tags: List[str]
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """문장과 띄어쓰기 tagging에 대해 feature로 변환한다.
        Args:
            sentence: 문장
            tags: 띄어쓰기 tagging
        Returns:
            feature를 리턴한다.
            input_ids, attention_mask, token_type_ids, slot_labels
        """

        input_tokens = []
        slot_labels = []

        for word, tag in zip(sentence, tags):
            tokens = self.tokenizer.tokenize(word)

            if len(tokens) == 0:
                tokens = self.tokenizer.unk_token

            input_tokens.extend(tokens)

            for i, _ in enumerate(tokens):
                if i == 0:
                    slot_labels.extend([tag])
                else:
                    slot_labels.extend([self.pad_token_id])

        # 2. max_len보다 길이가 길면 뒤에 자르기
        if len(input_tokens) > self.max_len - 2:
            input_tokens = input_tokens[: self.max_len - 2]
            slot_labels = slot_labels[: self.max_len - 2]

        # cls, sep 추가
        input_tokens = (
            [self.tokenizer.cls_token] + input_tokens + [self.tokenizer.sep_token]
        )
        slot_labels = [self.pad_token_id] + slot_labels + [self.pad_token_id]

        # token을 id로 변환
        input_ids = self.tokenizer.convert_tokens_to_ids(input_tokens)

        attention_mask = [1] * len(input_ids)
        token_type_ids = [0] * len(input_ids)

        # padding
        pad_len = self.max_len - len(input_tokens)
        input_ids = input_ids + ([self.tokenizer.pad_token_id] * pad_len)
        slot_labels = slot_labels + ([self.pad_token_id] * pad_len)
        attention_mask = attention_mask + ([0] * pad_len)
        token_type_ids = token_type_ids + ([0] * pad_len)

        input_ids = torch.tensor(input_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long)
        slot_labels = torch.tensor(slot_labels, dtype=torch.long)

        return input_ids, attention_mask, token_type_ids, slot_labels

## 모델 생성

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import BertConfig, BertModel, AdamW
from torch.utils.data import DataLoader
from seqeval.metrics import f1_score


class SpacingBertModel(pl.LightningModule):
    def __init__(
        self,
        config,
        ner_train_dataloader: DataLoader,
        ner_val_dataloader: DataLoader,
        ner_test_dataloader: DataLoader,
    ):
        super().__init__()
        self.config = config
        self.ner_train_dataloader = ner_train_dataloader,
        self.ner_val_dataloader = ner_val_dataloader,
        self.ner_test_dataloader = ner_test_dataloader
        self.slot_labels_type = load_slot_labels()
        self.pad_token_id = 0

        self.bert_config = BertConfig.from_pretrained(
            self.config.bert_model, num_labels=len(self.slot_labels_type)
        )
        self.model = BertModel.from_pretrained(
            self.config.bert_model, config=self.bert_config
        )
        self.dropout = nn.Dropout(self.config.dropout_rate)
        self.linear = nn.Linear(
            self.bert_config.hidden_size, len(self.slot_labels_type)
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        x = outputs[0]
        x = self.dropout(x)
        x = self.linear(x)

        return x

    def training_step(self, batch, batch_nb):

        input_ids = attention_mask = token_type_ids = slot_labels = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_labels)
        tensorboard_logs = {"train_loss": loss}

        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_nb):

        input_ids = attention_mask = token_type_ids = slot_labels = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        loss = self._calculate_loss(outputs, slot_labels)
        gt_slot_labels, pred_slot_labels = self._convert_ids_to_labels(
            outputs, slot_labels
        )

        val_acc = self._f1_score(gt_slot_labels, pred_slot_labels)

        return {"val_loss": loss, "val_acc": val_acc}

    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        val_acc = torch.stack([x["val_acc"] for x in outputs]).mean()

        tensorboard_log = {
            "val_loss": val_loss,
            "val_acc": val_acc,
        }

        return {"val_loss": val_loss, "progress_bar": tensorboard_log}

    def test_step(self, batch, batch_nb):

        input_ids = attention_mask = token_type_ids = slot_labels = batch

        outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        gt_slot_labels, pred_slot_labels = self._convert_ids_to_labels(
            outputs, slot_labels
        )

        test_acc = self._f1_score(gt_slot_labels, pred_slot_labels)

        test_step_outputs = {
            "test_acc": test_acc,
            "gt_labels": gt_slot_labels,
            "pred_labels": pred_slot_labels,
        }

        return test_step_outputs

    def test_epoch_end(self, outputs):
        test_acc = torch.stack([x["test_acc"] for x in outputs]).mean()

        gt_labels = []
        pred_labels = []
        for x in outputs:
            gt_labels.extend(x["gt_labels"])
            pred_labels.extend(x["pred_labels"])

        test_step_outputs = {
            "test_acc": test_acc,
            "gt_labels": gt_labels,
            "pred_labels": pred_labels,
        }

        return test_step_outputs

    def configure_optimizers(self):
        return AdamW(self.model.parameters(), lr=2e-5, eps=1e-8)

    def train_dataloader(self):
        return self.ner_train_dataloader

    def val_dataloader(self):
        return self.ner_val_dataloader

    def test_dataloader(self):
        return self.ner_test_dataloader

    def _calculate_loss(self, outputs, labels):
        # active_loss = attention_mask.view(-1) == 1
        # active_logits = outputs.view(-1, len(self.slot_labels_type))[active_loss]
        # active_labels = slot_labels.view(-1)[active_loss]
        active_logits = outputs.view(-1, len(self.slot_labels_type))
        active_labels = labels.view(-1)
        loss = F.cross_entropy(active_logits, active_labels)

        return loss

    def _f1_score(self, gt_slot_labels, pred_slot_labels):
        return torch.tensor(
            f1_score(gt_slot_labels, pred_slot_labels), dtype=torch.float32
        )

    def _convert_ids_to_labels(self, outputs, slot_labels):
        _, y_hat = torch.max(outputs, dim=2)
        y_hat = y_hat.detach().cpu().numpy()
        slot_label_ids = slot_labels.detach().cpu().numpy()

        slot_label_map = {i: label for i, label in enumerate(self.slot_labels_type)}
        slot_gt_labels = [[] for _ in range(slot_label_ids.shape[0])]
        slot_pred_labels = [[] for _ in range(slot_label_ids.shape[0])]

        for i in range(slot_label_ids.shape[0]):
            for j in range(slot_label_ids.shape[1]):
                if slot_label_ids[i, j] != self.pad_token_id:
                    slot_gt_labels[i].append(slot_label_map[slot_label_ids[i][j]])
                    slot_pred_labels[i].append(slot_label_map[y_hat[i][j]])

        return slot_gt_labels, slot_pred_labels

## 모델 학습

In [15]:
train_data_path = '/content/drive/MyDrive/news_class9x1400/train.txt'
test_data_path = '/content/drive/MyDrive/news_class9x1400/test.txt'
val_data_path = '/content/drive/MyDrive/news_class9x1400/val.txt'

In [48]:
from omegaconf import OmegaConf

config = OmegaConf.load('/content/drive/MyDrive/train_config.yaml')

In [49]:
def get_dataloader(
    data_path: str, transform: Callable[[List, List], Tuple], batch_size: int
) -> DataLoader:
    """dataloader 생성
    Args:
        data_path: dataset 경로
        transform: input feature로 변환해주는 funciton
        batch_size: dataloader batch size
    Returns:
        dataloader
    """
    dataset = CorpusDataset(data_path, transform)
    print(dataset[0])
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

In [ ]:
preprocessor = Preprocessor(config.max_len)
train_dataloader = get_dataloader(
    config.train_data_path, preprocessor.get_input_features, config.train_batch_size
)
val_dataloader = get_dataloader(
    config.val_data_path, preprocessor.get_input_features, config.train_batch_size
)
test_dataloader = get_dataloader(
    config.test_data_path, preprocessor.get_input_features, config.eval_batch_size
)

In [54]:
bert_finetuner = SpacingBertModel(
    config, 
    train_dataloader, 
    val_dataloader, 
    test_dataloader
)

In [55]:
import os
from pytorch_lightning.loggers import TensorBoardLogger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

logger = TensorBoardLogger(
    save_dir=os.path.join(config.log_path, config.task), version=1, name=config.task
)

checkpoint_callback = ModelCheckpoint(
    filepath="checkpoints/{epoch}_{val_loss:3f}",
    verbose=True,
    monitor="val_loss",
    mode="min",
    save_top_k=3,
    prefix="",
)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=3,
    verbose=False,
    mode="min",
)

In [53]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [56]:
# GPU 설정
device = torch.device("cpu")

In [59]:
trainer = pl.Trainer()

trainer.fit(bert_finetuner)
trainer.test()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name    | Type      | Params
--------------------------------------
0 | model   | BertModel | 92.2 M
1 | dropout | Dropout   | 0     
2 | linear  | Linear    | 5.4 K 
--------------------------------------
92.2 M    Trainable params
0         Non-trainable params
92.2 M    Total params
368.769   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

AttributeError: ignored

In [60]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"